# Методы борьбы с переобучением в нейронных сетях

Функция от весов, оптимизируемая при помощи алгоритма градиентного спуска, является сложной, невыпуклой, с большим количеством локальных экстремумов. Решая такую задачу, можно легко застрять в локальном минимуме, поэтому важно знать техники борьбы с переобучением.

### Регуляризация

Регуляризация - уже знакомый по классическому машинному обучению метод борьбы с переобучением. Он применяется к моделям, у которых слишком много свободных параметров, потому что они плохо обобщаются. В этом случае коэффициенты модели становятся слишком большими. Тогда логично добавить в оптимизируемую функцию штраф за слишком большие коэффициенты. С байесовской точки зрения многие методы регуляризации соответствуют добавлению некоторых априорных распределений на параметры модели.

Два наиболее распространенных типа регуляризации:

* $L_1$-регуляризация, или регуляризация через манхэттенское расстояние (lasso regression):

$$\theta^* = argmin_\theta (Q(\theta) + \lambda \sum\limits_{i,j,k} ||\theta_{ij}^{(k)}||)$$

* $L_2$-регуляризация, или регуляризация Тихонова (ridge regression):

$$\theta^* = argmin_\theta (Q(\theta) + \lambda \sum\limits_{i,j,k} ||\theta_{ij}^{(k)}||^2)$$

Также $L_2$-регуляризацию называют **weight decay** (сокращение весов).

$\lambda$ - коэффициент регуляризации, он задает то, насколько строго будет введено ограничение на параметры $w$, то есть чем больше коээфициент регуляризации, тем выше смещение и меньше разброс получившейся модели и наоборот.

### Early stopping

Самый простой и понятный метод борьбы с переобучением. Делим нашу обучающую выборку на train и valid, учим нашу нейронную сеть и сравниваем ошибки на каждом из наборов. Как только видим, что ошибка на valid начинает расти, останавливаем обучение.

<img src="pictures/train_valid.png" width=400 height=400 />

### Dropout (прореживание нейронной сети)

Dropout - тип регуляризации, характерный только для нейронных сетей. Основная идея, на которую опирается этот метод, - исключение некоторых нейронов (тип **drop neuron**) или связей между ними (тип **drop connection**) с целью снижения количества параметров нейронной сети. Для каждого элемента сети подбирается некоторое число (**dropout rate**) - вероятность, с которой нейрон или связь будут занулены, то есть, соответствующий выход будет или вычисляться "как обычно" при прямом или обратном распространении ошибки, или будет равен нулю.

Дропаут нельзя применять на последнем слое, так как нам нужно ожидаемое количество выходов из сети.

<img src="pictures/dropout.png" width=500 height=500 />

### Инициализация весов

Первое, что приходит в голову, когда встает вопрос о том, как правильно инициализировать веса, почему бы не инициализировать их нулями? Тогда все нейроны получат на вход одинаковые значения. Но нейронная сеть - последовательное преобразование признакового пространства, и мы наоборот хотим, чтобы входы и выходы были непохожи друг на друга. Вот и сделаем нейроны разными сразу на инициализации.

Самый простой и распространенный способ инициализации весов - небольшие случайные числа:
* $\theta^{(0)} \sim \text{norm}(0, \sigma^2)$
* $\theta^{(0)} \sim \text{random}(-\frac{1}{2n},\frac{1}{2n})$

Для скрытых слоев рекомендуется использовать инициализацию Ксавье (xavier initialization), где $n_{\text{in}}$ - число нейронов на текущем слое, $n_{\text{out}}$ – число нейронов на следующем слое: $\theta^{(0)} \sim [-\frac{\sqrt{6}}{\sqrt{n_{in} + n_{out}}}, \frac{\sqrt{6}}{\sqrt{n_{in} + n_{out}}}]$

Также во многих задачах начальное приближение весов некоторых отдельных слоев можно получить, применяя предобучение без учителя (unsupervised pretraining). Это хорошая практика, ведь хорошее начальное приближение поможет быстрее попасть в точку оптимума функции. Есть много способов того, как обучать начальное приближение:
* [Ограниченная машина Больцмана](https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%B0_%D0%91%D0%BE%D0%BB%D1%8C%D1%86%D0%BC%D0%B0%D0%BD%D0%B0#:~:text=%D0%9C%D0%B0%D1%88%D0%B8%CC%81%D0%BD%D0%B0%20%D0%91%D0%BE%CC%81%D0%BB%D1%8C%D1%86%D0%BC%D0%B0%D0%BD%D0%B0%20(%D0%B0%D0%BD%D0%B3%D0%BB.,%D1%81%D1%82%D0%BE%D1%85%D0%B0%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9%20%D0%B3%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D1%8B%D0%B9%20%D0%B2%D0%B0%D1%80%D0%B8%D0%B0%D0%BD%D1%82%20%D1%81%D0%B5%D1%82%D0%B8%20%D0%A5%D0%BE%D0%BF%D1%84%D0%B8%D0%BB%D0%B4%D0%B0.) - вероятностная модель, в которой есть известные (visible) переменные и переменные, значение которых необходимо вычислить (hidden), а также связи между ними. Задача заключается в востановлении скрытых переменных по входным данным и видимым переменным (восстанавливаем условное распределение на данных). В основе подхода лежит [метод Монте-Карло](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%9C%D0%BE%D0%BD%D1%82%D0%B5-%D0%9A%D0%B0%D1%80%D0%BB%D0%BE).
* Автокодировщики (autoencoders)
* И т.д.

### Нормализация батчей

Мини-батч градинтный спуск - одна из наиболее часто употребимых на практике модификаций градиентного спуска, в которой множество объектов батча выбирается случайным образом.

Однако, в случае с многослойной нейронной сетью мы можем столкнуться с проблемой, которая называется **ковариантный сдвиг (covariance shift)**: если распределение данных от батча к батчу не очень похоже для одного из первых слоев нейронной сети (то есть веса имеют разные параметры: математическое ожидание, дисперсия и прочее), это может привести к изменению активаций всех последующих слоев сети. Необходимо добиться того, чтобы батчи были похожи, то есть **нормализовать данные в батче**. Как правильно это сделать?

Можно масштабировать данные, которые подаются на вход нейронной сети при помощи скейлера. Важно следить за дисбалансом классов в выборке.

Также необходимо выполнять нормализацию весов в скрытых слоях нейронной сети, что и является основной идеей **batch normalization**: входы слоя должны быть "обелены" (whitened), то есть их среднее приведено к нулю, а матрица ковариаций - к единичной. Это уменьшит величины, на которые смещаются значения узлов в скрытых слоях, поможет победить ковариантный сдвиг и решить проблему взрывающегося градиента, ускорит процесс обучения. При этом у каждого слоя должна быть своя нормализация.

> Ковариация случайных величин -  мера зависимости двух случайных величин. Матрица ковариаций - это матрица, составленная из попарных ковариаций элементов одного или двух случайных векторов. 

"Обеление" происходит в алгоритме градиентного спуска **на каждом слое по отдельности**. Его идея заключается в том, чтобы отмасштабировать данные так, чтобы стандартное отклонение по всем параметрам было одинаковым (равным единице).

Алгоритм нормализации на каждом отдельном слое нейронной сети выглядит так:
* Получаем некоторый батч данных $\chi$ ($|\chi| = N$) и некоторый вектор $x_i$ из обучающей выборки
* Прогоняем батч через слой нейронной сети, получая при этом выходы из этого слоя, и вычисляем:
    * Среднее значение по выходам из батча: $\mu = \frac{1}{N} \sum_{i=1}^N x_i$
    * Стандартное отклонение по выходам из батча: $\sigma^2 = \sum_{i=1}^N (x_i - \mu)^2$
* Нормализуем данные в батче: $\hat{x_i} = \gamma \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta$, где 
    * $\epsilon$ - небольшая константа
    * $\gamma$ и $\beta$ - обучаемые параметры слоя нормализации
* Повторяем то же самое для остальных батчей (среднее значение и стандартные отклонения на каждом батче будут разными)
* Финальные значения параметров $\gamma$ и $\beta$ для инференса рассчитываются при помощи экспоненциального скользящего среднего

Когда стоит делать нормализацию?
* После очередного слоя
* После линейной части слоя
* До нелинейной функции активации

В pytorch:
* nn.BatchNorm1d (nn.BatchNorm2d, nn.BatchNorm3d)
* nn.LazyBatchNorm1d (nn.LazyBatchNorm2d, nn.LazyBatchNorm3d)
* nn.LayerNorm
* nn.GroupNorm
* nn.SyncBatchNorm
* nn.LocalResponseNorm
* nn.InstanceNorm1d (nn.InstanceNorm2d, nn.InstanceNorm3d)
* nn.LazyInstanceNorm1d (nn.LazyInstanceNorm2d, nn.LazyInstanceNorm3d)